In [2]:
from repsheet_backend.common import db_connect, MEMBER_VOTES_TABLE, VOTES_HELD_TABLE
from repsheet_backend.bills import download_all_bill_texts, BillId
from multiprocessing.pool import ThreadPool

## Enumerate every bill voted on by a current member

In [3]:
with db_connect() as db:
    bills = db.execute(
        "SELECT DISTINCT v.[Parliament], v.[Session], v.[Bill Number] "
        f"FROM {MEMBER_VOTES_TABLE} mv "
        f"LEFT JOIN {VOTES_HELD_TABLE} v ON v.[Vote ID] = mv.[Vote ID] "
        "WHERE [Bill Number] IS NOT NULL "
        "ORDER BY v.[Parliament] DESC "
    ).fetchall()

    bills = [BillId(*row) for row in bills]
    with ThreadPool(8) as pool:
        results = pool.starmap(download_all_bill_texts, bills)

Note that we are missing a bunch of bills from past parliaments as they have been split into multiple pages, with one XML file per page.

In [4]:
# Check if we have all bill texts for the latest session of parliament
missing, found = 0, 0
for bill, is_bound in zip(bills, results):
    parliament, session, bill_number = bill
    if not is_bound:
        print(f"Bill text not found for {bill}")
        missing += 1
    else:
        found += 1
missing, found

(0, 576)